# Intro. to Context Aware Snorkel: Extracting Spouse Relations from the News Across Sentences

## Part III: Training an End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a Bi-LSTM, a state-of-the-art deep neural network implemented in [TensorFlow](https://www.tensorflow.org/).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass:

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

We reload the probabilistic training labels:

In [3]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

We also reload the candidates:

In [4]:
train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

Finally, we load gold labels for evaluation:

In [5]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

Now we can setup our discriminative model. Here we specify the model and learning hyperparameters.

They can also be set automatically using a search based on the dev set with a [GridSearch](https://github.com/HazyResearch/snorkel/blob/master/snorkel/learning/utils.py) object.

In [6]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':            0.01,
    'embedding_dim': 50,
    'hidden_dim':    50,
    'n_epochs':      16,
    'dropout':       0.25,
    'seed':          1701
}

lstm = LSTM(n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[LSTM] Training model
[LSTM] n_train=10669  #epochs=16  batch size=64
[LSTM] Epoch 1 (23.47s)	Average loss=0.527465	Dev F1=0.00
[LSTM] Epoch 2 (49.47s)	Average loss=0.503755	Dev F1=28.29
[LSTM] Epoch 3 (75.27s)	Average loss=0.490142	Dev F1=25.40
[LSTM] Epoch 4 (100.27s)	Average loss=0.481050	Dev F1=18.79
[LSTM] Epoch 5 (125.83s)	Average loss=0.474643	Dev F1=33.66
[LSTM] Epoch 6 (152.43s)	Average loss=0.469516	Dev F1=30.86
[LSTM] Epoch 7 (177.55s)	Average loss=0.466049	Dev F1=35.29
[LSTM] Epoch 8 (201.65s)	Average loss=0.463393	Dev F1=29.71
[LSTM] Epoch 9 (226.02s)	Average loss=0.461703	Dev F1=35.45
[LSTM] Epoch 10 (250.61s)	Average loss=0.460300	Dev F1=37.50
[LSTM] Epoch 11 (273.93s)	Average loss=0.458412	Dev F1=41.28
[LSTM] Epoch 12 (298.12s)	Average loss=0.457409	Dev F1=32.32
[LSTM] Epoch 13 (322.07s)	Average loss=0.456363	Dev F1=38.38
[LSTM] Epoch 14 (345.89s)	Average loss=0.455036	Dev F1=34.42
[LSTM] Model saved as <LSTM>
[LSTM] Epoch 15 (369.11s)	Average loss=0.454661	Dev F1=43.27

Now, we get the precision, recall, and F1 score from the discriminative model:

In [7]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.491, Recall: 0.568, F1 Score: 0.527


We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [8]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test, set_unlabeled_as_neg=False)

Scores (Un-adjusted)
Pos. class accuracy: 0.568
Neg. class accuracy: 0.933
Precision            0.506
Recall               0.568
F1                   0.535
----------------------------------------
TP: 83 | FP: 81 | TN: 1125 | FN: 63



In [9]:
for metric in [tp, fp, tn, fn]:
    count = 0
    for sample in metric:
        if sample.person1.sentence.position != sample.person2.sentence.position:
            count += 1
    print(str(count))

0
0
0
0


Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

You can also improve performance substantially by increasing the number of training epochs!

Finally, we can save the predictions of the model on the test set back to the database. (This also works for other candidate sets, such as unlabeled candidates.)

In [10]:
lstm.save_marginals(session, test_cands)

Saved 1417 marginals


##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!